In [1]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
gym_data = pd.read_csv('crowdness_gym_data.csv')

In [4]:
# Helper function to check dataset compatibility with SGDRegressor
def check_and_fix_dataframe_dtypes(X):
    """Attempts to convert all features in a DataFrame to numeric for SGD Regressor."""
    X_numeric = X.apply(pd.to_numeric, errors='coerce')  # Attempt to convert to numeric, set invalid values to NaN
    non_numeric_columns = X.columns[X_numeric.isna().any()].tolist()  # Identify columns with non-numeric entries
    X_cleaned = X_numeric.dropna(axis=1)  # Drop columns with non-numeric entries
    if X_numeric.shape[1] != X_cleaned.shape[1]:
        print("Some non-numeric columns were dropped:")
        print(non_numeric_columns)
    else:
        print("All features are now numeric and compatible with SGD Regressor.")
    return X_cleaned

# Check and fix the feature dtypes
gym_data_ready = check_and_fix_dataframe_dtypes(gym_data)

Some non-numeric columns were dropped:
['date']


In [5]:
# Make a pipeline that scales and uses SGDRegressor to fit the data
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('sgd_regressor', SGDRegressor())
])

# Define features and target variable
# Assuming gym_data has columns 'features' for independent variables and 'target' for the dependent variable
X = gym_data_ready.drop(columns=['number_people'])  # Adjust 'number_people' to match the actual column name
y = gym_data_ready['number_people']

# Fix dataframe dtypes for compatibility
X = check_and_fix_dataframe_dtypes(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")

All features are now numeric and compatible with SGD Regressor.
Mean Squared Error: 251.03041289356534
Mean Absolute Error: 12.110099119631247
R2 Score: 0.5143048655317342


In [6]:
# Perform hyperparameter search using GridSearchCV
from sklearn.model_selection import GridSearchCV

# Define parameters for hyperparameter search
param_grid = {
    'sgd_regressor__alpha': [0.0001, 0.001, 0.01, 0.1],
    'sgd_regressor__penalty': ['l2', 'l1', 'elasticnet'],
    'sgd_regressor__eta0': [0.01, 0.1, 0.5],
    'sgd_regressor__max_iter': [1000, 2000, 3000]
}

# Wrap the pipeline with GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Evaluate the best model
print(f"Best Parameters: {grid_search.best_params_}")
y_pred = grid_search.best_estimator_.predict(X_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")

C:\Users\User\Dataquest.IO\Guided Project - Popular Data Science Questions\GuidedProjectDataScienceQuestions\venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\User\Dataquest.IO\Guided Project - Popular Data Science Questions\GuidedProjectDataScienceQuestions\venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Best Parameters: {'sgd_regressor__alpha': 0.0001, 'sgd_regressor__eta0': 0.01, 'sgd_regressor__max_iter': 3000, 'sgd_regressor__penalty': 'l2'}
Mean Squared Error: 251.1063612387778
Mean Absolute Error: 12.106971907943624
R2 Score: 0.5141579202221384
